In [28]:
import sqlite3
import pandas as pd

In [29]:
con = sqlite3.connect(':memory:')

In [30]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [31]:
for table in table_names:
    data = pd.read_csv(f'D:\\DS\\DS school\\sql\\{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [32]:
tasks = 0

## Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [33]:
tasks += 1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM Customers where Country in ('Germany', 'France') or City is 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [34]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, count(CustomerID) as count FROM Customers group by Country order by count desc limit 3
    ''',
    con,
)

,Country,count
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [61]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate FROM Orders as o join Shippers as s on o.ShipperID = s.ShipperID 
    order by o.OrderID limit 1 offset 9
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [36]:
tasks+=1
pd.read_sql(
    '''
    SELECT p.ProductName, p.Price  FROM OrderDetails as od 
    join Products as p on p.ProductID = od.ProductID 
    where OrderID = 
    (
    SELECT OrderID FROM OrderDetails as od 
    join Products as p on od.ProductID = p.ProductID group by OrderID order by sum(OrderID) desc limit 1
    )
    ''',
    con,
)

,ProductName,Price
0,Chais,18.0
1,Sir Rodney's Scones,10.0
2,Rössle Sauerkraut,45.6
3,Inlagd Sill,19.0
4,Boston Crab Meat,18.4


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [37]:
tasks+=1
pd.read_sql(
    '''
    select p.ProductName, od.OrderID, od.Quantity from Products as p
    join OrderDetails as od on p.ProductID = od.ProductID
    where p.ProductID = 
    (
    SELECT ProductID FROM OrderDetails group by ProductID order by sum(Quantity) desc limit 1
    )
    ''',
    con,
)

,ProductName,OrderID,Quantity
0,Gorgonzola Telino,10253,20
1,Gorgonzola Telino,10272,40
2,Gorgonzola Telino,10273,15
3,Gorgonzola Telino,10325,4
4,Gorgonzola Telino,10335,25
5,Gorgonzola Telino,10342,56
6,Gorgonzola Telino,10356,30
7,Gorgonzola Telino,10359,70
8,Gorgonzola Telino,10363,20
9,Gorgonzola Telino,10374,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [38]:
tasks+=1
pd.read_sql(
    '''
    select SupplierName, Country, ContactName, Phone from Suppliers
    where SupplierID in 
    (
    SELECT p.SupplierID FROM OrderDetails as od
    join Products as p on p.ProductID = od.ProductID
    group by SupplierID
    order by count(od.OrderID) desc limit 5
    )
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
2,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
3,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
4,Norske Meierier,Norway,Beate Vileid,(0)2-953010


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [39]:
tasks+=1
pd.read_sql(
    '''
    select 'Brazil' as Country, c.CategoryName, (p.Price*od.Quantity) as total_price 
    from OrderDetails as od
    join Products as p on p.ProductID = od.ProductID
    join Categories as c on p.CategoryID = c.CategoryID
    where OrderID in 
    (
    select OrderID from Orders where CustomerID in 
    (select CustomerID from Customers where Country = 'Brazil')
    )
    order by total_price desc limit 1
    ''',
    con,
)

,Country,CategoryName,total_price
0,Brazil,Beverages,10540.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [40]:
tasks+=1
pd.read_sql(
    '''
    select (max(total_price)-(min(total_price))) as diff from
    (
    select od.OrderID, sum(p.Price*od.Quantity) as total_price from OrderDetails as od
    join Products as p on p.ProductID = od.ProductID
    where od.OrderID in
    (
    select OrderID from Orders where CustomerID in 
    (select CustomerID from Customers where Country = 'USA')   
    )
    group by od.OrderID order by total_price
    )
    ''',
    con,
)

,diff
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

Возникли вопросы с обработкой Даты, почему-то часть данных определяет как NaT. Может эту проблему можно решить как-то быстро и в одну строку, но я не нашел такой способ. Пришлось немного расписать.

In [41]:
data = pd.read_sql(
    '''
    select EmployeeID, BirthDate from Employees
    ''',
    con, parse_dates=["BirthDate"]
)

In [42]:
data

,EmployeeID,BirthDate
0,1,1968-08-12
1,2,NaT
2,3,NaT
3,4,NaT
4,5,1955-04-03
5,6,1963-02-07
6,7,NaT
7,8,1958-09-01
8,9,1969-02-07
9,10,NaT


In [43]:
data = pd.read_sql(
    '''
    select EmployeeID, BirthDate from Employees
    ''',
    con,
)

In [44]:
data

,EmployeeID,BirthDate
0,1,08.12.1968
1,2,19.02.1952
2,3,30.08.1963
3,4,19.09.1958
4,5,04.03.1955
5,6,02.07.1963
6,7,29.05.1960
7,8,09.01.1958
8,9,02.07.1969
9,10,19.09.1928


In [45]:
data['BirthDate'] = data['BirthDate'].str.split('.')
data['day'] = data['BirthDate'].apply(lambda x: x[0])
data['month'] = data['BirthDate'].apply(lambda x: x[1])
data['year'] = data['BirthDate'].apply(lambda x: x[2])

In [46]:
data

,EmployeeID,BirthDate,day,month,year
0,1,"[08, 12, 1968]",08,12,1968
1,2,"[19, 02, 1952]",19,02,1952
2,3,"[30, 08, 1963]",30,08,1963
3,4,"[19, 09, 1958]",19,09,1958
4,5,"[04, 03, 1955]",04,03,1955
5,6,"[02, 07, 1963]",02,07,1963
6,7,"[29, 05, 1960]",29,05,1960
7,8,"[09, 01, 1958]",09,01,1958
8,9,"[02, 07, 1969]",02,07,1969
9,10,"[19, 09, 1928]",19,09,1928


In [47]:
data = data.drop('BirthDate', axis=1)

In [48]:
data

,EmployeeID,day,month,year
0,1,08,12,1968
1,2,19,02,1952
2,3,30,08,1963
3,4,19,09,1958
4,5,04,03,1955
5,6,02,07,1963
6,7,29,05,1960
7,8,09,01,1958
8,9,02,07,1969
9,10,19,09,1928


In [49]:
data.to_sql("Employees_work", con, if_exists = 'replace', index = False)

10

In [50]:
pd.read_sql(
    '''
   select EmployeeID from Employees_work order by year desc, month desc, day desc limit 3
    ''',
    con,
)

,EmployeeID
0,9
1,1
2,3


In [51]:
tasks+=1
pd.read_sql(
    '''
   select count(o.OrderID) as orders, (e.FirstName || ' ' || e.LastName) as Employer from Orders as o 
   join Employees as e on e.EmployeeID = o.EmployeeID
   where o.EmployeeID in 
   (
   select EmployeeID from Employees_work order by year desc, month desc, day desc limit 3
   )
   group by o.EmployeeID
    ''',
    con,
)

,orders,Employer
0,29,Nancy Davolio
1,31,Janet Leverling
2,6,Anne Dodsworth


10. Сколько банок крабового мяса всего было заказано.

In [52]:
tasks+=1
pd.read_sql(
    '''
    select sum(Quantity) as sum from OrderDetails where ProductID = 
    (
    select ProductID from Products where ProductName like '%crab%'
    )
    ''',
    con,
)

,sum
0,256


In [53]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
